In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!ls gdrive/My\ Drive/real-estate

In [0]:
!cp -r gdrive/My\ Drive/real-estate .

In [0]:
!ls real-estate

In [0]:
import os
import glob
import re
from pathlib import Path

import numpy as np
import pandas as pd

In [0]:
glob.glob('real-estate/*')

In [0]:
filenames = glob.glob(str(Path() / 'real-estate' / '*'))
filenames

In [0]:
sorted(filenames)

In [0]:
sorted_filenames = sorted(
    filenames,
    key=lambda s: int(s[:-5].split('_')[-1]),
)
sorted_filenames

In [0]:
price1 = pd.read_excel(sorted_filenames[0], header=16)
price1.head()

In [0]:
price1.info()

In [0]:
price1 = pd.read_excel(
    sorted_filenames[0],
    header=16,
    usecols=[
        '시군구',
        '주택유형',
        '도로조건',
        '연면적(㎡)',
        '대지면적(㎡)',
        '계약년월',
        '거래금액(만원)',
        '건축년도',
    ],
)
price1.columns = [
    'si_gun_gu',
    'housing_type',
    'distance_to_road',
    'total_floor_area',
    'plottage',
    'date_of_contract',
    'price',
    'construction_year'
]
price1.head()

In [0]:
price1 = pd.read_excel(
    sorted_filenames[0],
    header=16,
    usecols=[
        '시군구',
        '주택유형',
        '도로조건',
        '연면적(㎡)',
        '대지면적(㎡)',
        '계약년월',
        '거래금액(만원)',
        '건축년도',
    ],
    thousands=',',
)
price1.columns = [
    'si_gun_gu',
    'housing_type',
    'distance_to_road',
    'total_floor_area',
    'plottage',
    'date_of_contract',
    'price',
    'construction_year'
]
price1.head()
price1['price'].dtype

In [0]:
price1['construction_year'].describe()

In [0]:
price1['construction_year'].nsmallest(5)

In [0]:
price1.loc[price1['construction_year'] >= 1900, 'construction_year'].plot.hist()

In [0]:
(
    price1[price1['construction_year'] > 1900]
    .dropna()
    .astype({'construction_year': int})
).head()

In [0]:
price1['distance_to_road'].unique()

In [0]:
price1['distance_to_road'].replace(
    ['8m미만', '12m미만', '25m미만', '-', '25m이상'],
    [4.0, 10.0, 18.5, np.nan, 50.0],
).unique()

In [0]:
price1['date_of_contract'].head()

In [0]:
pd.to_datetime(price1['date_of_contract'].astype(str), format='%Y%m').head()

In [0]:
price1['housing_type'].unique()

In [0]:
price1['housing_type'].astype('category').head()

In [0]:
print(price1['housing_type'].memory_usage())
print(price1['housing_type'].astype('category').memory_usage())

In [0]:
prices = []
for filename in filenames:
    prices.append(pd.read_excel(
        filename,
        header=16,
        usecols=[
            '시군구',
            '주택유형',
            '도로조건',
            '연면적(㎡)',
            '대지면적(㎡)',
            '계약년월',
            '거래금액(만원)',
            '건축년도',
        ],
        thousands=',',
        dtype={
            '주택유형': 'category',
        }
    ).dropna())
price = pd.concat(prices).reset_index(drop=True)

price.columns = [
    'si_gun_gu',
    'housing_type',
    'distance_to_road',
    'total_floor_area',
    'plottage',
    'date_of_contract',
    'price',
    'construction_year'
]
price = price[price['construction_year'] > 1900].astype(
    {'construction_year': int}
)
price['distance_to_road'] = price['distance_to_road'].replace(
    ['8m미만', '12m미만', '25m미만', '-', '25m이상'],
    [4.0, 10.0, 18.5, np.nan, 50.0],
)
price = price.dropna().reset_index(drop=True)
price['date_of_contract'] = pd.to_datetime(
    price['date_of_contract'].astype(str),
    format='%Y%m'
)

In [0]:
price.head()

In [0]:
price.describe()

In [0]:
si_gun_gu = price['si_gun_gu'].str.split(expand=True)
si_gun_gu.head()

In [0]:
si_gun_gu.notna().sum()

In [0]:
si_gun_gu = price['si_gun_gu'].str.split(n=2, expand=True)
si_gun_gu.head()

In [0]:
price.info()

In [0]:
si_gun_gu.index[si_gun_gu[2].isna()]

In [0]:
si_gun_gu.isna().sum()

In [0]:
price.loc[60005]

In [0]:
si_gun_gu = si_gun_gu.fillna('')
si_gun_gu.columns = ['sido', 'sigungu', 'dongli']
si_gun_gu.head()

In [0]:
price = pd.concat([price, si_gun_gu], axis=1).drop(columns='si_gun_gu')
price.head()

# Visualization

In [0]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [0]:
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
pd.plotting.register_matplotlib_converters()

fm._rebuild()
mpl.rc('font', family='NanumGothic')

In [0]:
price['housing_type'].value_counts()

In [0]:
fig, axes = plt.subplots(1, 2)
price['housing_type'].value_counts().plot.pie(
    autopct='%.2f %%',
    ax=axes[0]
)
price['housing_type'].value_counts().plot.bar(ax=axes[1])
fig.tight_layout()

In [0]:
fig, ax = plt.subplots()
price['price'].plot.hist(ax=ax)
# price.plot.hist(y='price', ax=ax)
ax.set(
    xlabel='거래 금액(만 원)',
    ylabel='거래 건수',
)

In [0]:
price['price'].nlargest(10)

In [0]:
price[price['price'] > 100_0000]

In [0]:
fig, ax = plt.subplots()
price[price['price'] < 25_0000].plot.hist(y='price', ax=ax)
ax.set(
    xlabel='거래 금액(만 원)',
    ylabel='거래 건수',
)

In [0]:
fig, ax = plt.subplots()
price[price['price'] > 250_000].plot.hist(y='price', ax=ax)
ax.set(
    xlabel='거래 금액(만 원)',
    ylabel='거래 건수',
)

In [0]:
price_lower = price[price['price'] < 25_0000]

In [0]:
fig, ax = plt.subplots()

price_lower['date_of_contract'].dt.month.value_counts()

(
    price_lower['date_of_contract']
    .dt.month
    .value_counts()
    .sort_index()
    .plot.bar()
)

In [0]:
pd.plotting.scatter_matrix(
    price_lower[['price', 'construction_year', 'total_floor_area', 'plottage']],
    figsize=(10, 10),
)